## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
from src.database.database_utils import DatabaseManager

db = DatabaseManager()
print('Database connected')

## 1. Prepare Data for Modeling

In [ ]:
# Fetch data
df = db.query_accidents(limit=5000)
print(f'Data shape: {df.shape}')
print(f'\nColumns: {df.columns.tolist()}')

In [ ]:
# Create target: is it a fatal accident? (gravite_max = 4)
df['is_fatal'] = (df['gravite_max'] == 4).astype(int)

print(f'Fatal accidents: {df["is_fatal"].sum()} ({df["is_fatal"].mean()*100:.1f}%)')
print(f'Non-fatal: {(1-df["is_fatal"]).sum()} ({(1-df["is_fatal"].mean())*100:.1f}%)')

## 2. Feature Engineering

In [ ]:
# Create features
df['is_night'] = ((df['heure'] >= 22) | (df['heure'] <= 5)).astype(int)
df['is_weekend'] = df['jour_semaine'].isin(['Saturday', 'Sunday']).astype(int)
df['season'] = pd.cut(df['mois'], bins=[0, 3, 6, 9, 12], labels=['Winter', 'Spring', 'Summer', 'Fall'])

# Encode categorical
le = LabelEncoder()
df['season_encoded'] = le.fit_transform(df['season'].fillna('Unknown'))

print('Features created:')
print(df[['is_night', 'is_weekend', 'season']].head(10))

## 3. Predictive Model - Fatal Accident Prediction

In [ ]:
# Select features
features = ['heure', 'mois', 'is_night', 'is_weekend', 'season_encoded', 'nombre_personnes']
X = df[features].fillna(0)
y = df['is_fatal']

print(f'Features: {features}')
print(f'X shape: {X.shape}, y shape: {y.shape}')

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Training set: {X_train.shape}')
print(f'Test set: {X_test.shape}')

In [ ]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Predictions
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

print('Model trained!')

### Model Performance

In [ ]:
print('Classification Report:')
print(classification_report(y_test, y_pred, target_names=['Non-Fatal', 'Fatal']))

print(f'\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba):.3f}')

### Feature Importance

In [ ]:
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': importances
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 5))
plt.barh(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance - Fatal Accident Prediction')
plt.tight_layout()
plt.show()

print(feature_importance_df)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Fatal', 'Fatal'],
            yticklabels=['Non-Fatal', 'Fatal'])
plt.ylabel('True')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

## 4. Risk Scoring by Location

In [ ]:
df_danger = db.get_danger_scores(limit=50)

# Risk categories
df_danger['risk_category'] = pd.cut(df_danger['score_danger'], 
                                     bins=[0, 30, 50, 100], 
                                     labels=['Low', 'Medium', 'High'])

print('Risk distribution:')
print(df_danger['risk_category'].value_counts())
print('\nTop 10 high-risk communes:')
print(df_danger[df_danger['risk_category'] == 'High'][['nom_com', 'score_danger', 'nombre_accidents']].head(10))

## 5. Temporal Patterns Analysis

In [ ]:
df_temps = db.get_stats_temporelles()

# Hourly patterns
hourly = df_temps.groupby('heure').agg({
    'nombre_accidents': 'sum',
    'nombre_deces': 'sum',
    'gravite_moyenne': 'mean'
}).reset_index()

hourly['mortalite_rate'] = hourly['nombre_deces'] / hourly['nombre_accidents']

print('Hourly statistics:')
print(hourly.sort_values('mortalite_rate', ascending=False).head(10))

### Visualize hourly patterns

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Accidents by hour
axes[0, 0].plot(hourly['heure'], hourly['nombre_accidents'], marker='o')
axes[0, 0].set_title('Accidents by Hour')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Count')
axes[0, 0].grid(True, alpha=0.3)

# Deaths by hour
axes[0, 1].plot(hourly['heure'], hourly['nombre_deces'], marker='o', color='red')
axes[0, 1].set_title('Deaths by Hour')
axes[0, 1].set_xlabel('Hour')
axes[0, 1].set_ylabel('Count')
axes[0, 1].grid(True, alpha=0.3)

# Mortality rate
axes[1, 0].plot(hourly['heure'], hourly['mortalite_rate']*100, marker='o', color='darkred')
axes[1, 0].set_title('Mortality Rate by Hour')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Mortality Rate (%)')
axes[1, 0].grid(True, alpha=0.3)

# Average severity
axes[1, 1].plot(hourly['heure'], hourly['gravite_moyenne'], marker='o', color='orange')
axes[1, 1].set_title('Average Severity by Hour')
axes[1, 1].set_xlabel('Hour')
axes[1, 1].set_ylabel('Severity Score')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Anomaly Detection

In [ ]:
from sklearn.covariance import EllipticEnvelope

# Prepare data for anomaly detection
X_anom = df[['heure', 'mois', 'nombre_personnes']].fillna(0)

# Train anomaly detector
detector = EllipticEnvelope(contamination=0.05, random_state=42)
anomalies = detector.fit_predict(X_anom)

df['is_anomaly'] = (anomalies == -1).astype(int)

print(f'Anomalies detected: {df["is_anomaly"].sum()} ({df["is_anomaly"].mean()*100:.1f}%)')
print('\nCharacteristics of anomalies:')
print(df[df['is_anomaly'] == 1][['heure', 'mois', 'nombre_personnes', 'gravite_max']].describe())

## 7. Key Insights & Recommendations

In [ ]:
print('=== KEY FINDINGS ===')
print()
print('1. MOST DANGEROUS HOURS:')
peak_hours = hourly.nlargest(5, 'nombre_accidents')[['heure', 'nombre_accidents']]
print(peak_hours.to_string(index=False))
print()

print('2. HIGHEST MORTALITY HOURS:')
deadly_hours = hourly.nlargest(5, 'mortalite_rate')[['heure', 'mortalite_rate']]
print(deadly_hours.to_string(index=False))
print()

print('3. MODEL PREDICTIONS:')
print(f'   - Accuracy: {(y_pred == y_test).mean():.1%}')
print(f'   - Most important feature: {feature_importance_df.iloc[0]["feature"]}')
print()

print('4. RECOMMENDATIONS:')
print('   - Increase enforcement during peak accident hours')
print('   - Focus on high-risk communes identified by danger scores')
print('   - Implement safety measures in areas with anomalous patterns')

## 8. Export Analysis Results

In [ ]:
# Save results
feature_importance_df.to_csv('feature_importance.csv', index=False)
df_danger.to_csv('danger_scores.csv', index=False)
hourly.to_csv('hourly_analysis.csv', index=False)

print('Results exported:')
print('  - feature_importance.csv')
print('  - danger_scores.csv')
print('  - hourly_analysis.csv')